In [ ]:
# # uncomment to install segment anything and other dependencies
# !pip install git+https://github.com/facebookresearch/segment-anything.git
# !pip install opencv-python pycocotools matplotlib onnxruntime onnx

In [ ]:
# import necessary libraries
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from segment_anything import sam_model_registry, SamPredictor
import os
import gc
import tqdm

In [ ]:
# Define variables
SAM_CHECKPOINT = os.path.join('..','SAM model','sam_vit_l_0b3195.pth')
MODEL_TYPE = 'vit_l'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# Load the model
sam_model = sam_model_registry[MODEL_TYPE](checkpoint=SAM_CHECKPOINT)
sam_model = sam_model.to(DEVICE)
sam_promt2mask = SamPredictor(sam_model)

In [ ]:
# Load the dataset
benign_dataframe_path = os.path.join('..','Dataset_BUSI_with_GT','benign_bounding_box.csv')
malignant_dataframe_path = os.path.join('..','Dataset_BUSI_with_GT','malignant_bounding_box.csv')

benign_dataframe = pd.read_csv(benign_dataframe_path)
malignant_dataframe = pd.read_csv(malignant_dataframe_path)

In [ ]:
# visualize the benign dataframe
benign_dataframe.head()

In [ ]:
from segment_anything.utils.transforms import ResizeLongestSide
resize_transform = ResizeLongestSide(sam_model.image_encoder.img_size)

# prepare image by resizing and converting to tensor
def prepare_image(image, transform, model):
    image = transform.apply_image(image)
    image = torch.as_tensor(image, device=model.device) 
    return image.permute(2, 0, 1).contiguous()

# define to prepare batch of images
def batch_dataloader(data_frame, batch_size, prompt , shuffle=True, model=sam_model, random_state=None):
    batch = []
    indices = []
    # shuffle the data
    if shuffle:
        data_frame = data_frame.sample(frac=1, random_state=random_state)

    for i, row in data_frame.iterrows():
        img_path = os.path.join('..',row['path'])
        # read the image and convert to RGB
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        og_size = image.shape[:2]

        # prepare the image
        image = prepare_image(image, resize_transform, model)

        # read the bounding box
        bbox = torch.tensor([row['x_min'], row['y_min'], row['x_max'], row['y_max']]).view(1, 4)
        bbox = bbox.to(model.device)
        bbox = resize_transform.apply_boxes_torch(bbox, og_size)

        if prompt =='BBox+Point' or prompt =='Point':
            # read point prompts
            point_coords = torch.tensor([row['center_x'], row['center_y']]).view(1, 1, 2)
            point_coords = resize_transform.apply_coords_torch(point_coords, og_size)  
            point_coords = point_coords.to(model.device)

            point_labels = torch.tensor(1).view(1, 1)
            point_labels = point_labels.to(model.device)
            
            if prompt =='BBox+Point':
                # create dictionary
                img_dict = {
                    'image': image, 
                    'boxes': bbox,
                    'original_size': og_size,
                    'point_coords': point_coords,
                    'point_labels': point_labels,
                }
            
            else:
                # create dictionary
                img_dict = {
                    'image': image,
                    'point_coords': point_coords,
                    'point_labels': point_labels,
                    'original_size': og_size,
                }
            
        elif prompt == 'BBox':
            # create dictionary
            img_dict = {
                'image': image,
                'boxes': bbox,
                'original_size': og_size
            }
        else:
            raise ValueError('prompt should be either BBox, BBox+Point or Point')
        
        indices.append(i)
        batch.append(img_dict)

        if len(batch) == batch_size:
            yield indices,batch
            batch = []
            indices = []


In [ ]:
# check if images, boxes and points are transformed correctly
import matplotlib.patches as patches
for batch,item in batch_dataloader(malignant_dataframe, 1, 'BBox+Point', shuffle=True, random_state=None):
    item=item[0]

    img = item['image'].cpu()
    boxes = item['boxes']
    point_coords = item['point_coords'].cpu().numpy().squeeze().squeeze()
    point_labels = item['point_labels']

    fig,ax = plt.subplots(1)
    ax.imshow(img.permute(1, 2, 0))
    for box in boxes:
        box = box.cpu().numpy()
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.scatter(point_coords[0],point_coords[1])
    ax.axis('off')
    plt.show()
    break

In [ ]:
def clear_vram():
    if DEVICE == 'cuda':
        torch.cuda.empty_cache()
        gc.collect()
    else:
        pass

In [ ]:
# define function to unpack batched output and save the masks
def unpack_and_save_op(indices, batch_op, data_frame,save_dir):
    '''
    indices: list of indices in original data_frame
    batch_op: list of output dictionaries
    data_frame: original data_frame
    save_dir: directory to save the output masks
    '''
    # Check if save_dir exists, create it if it doesn't
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for i, index in enumerate(indices):
        # get the row for index
        row = data_frame.iloc[index]

        # get the output dictionary
        op_dict = batch_op[i]

        # get the mask
        mask = op_dict['masks'].squeeze().cpu().numpy()

        # save mask to given directory with name in dafaframe with added prediction word
        mask_path = os.path.join(save_dir, row['image'].replace('mask.png', 'mask_prediction.png'))

        # save the mask
        plt.imsave(mask_path, mask, cmap='gray')

        # clear the memory
        del op_dict
        clear_vram()

In [ ]:
# clear vram before starting
clear_vram()

## only BBox prompt

In [ ]:
path_to_save = os.path.join('SAM_predictions_BBox','benign')
with tqdm.tqdm(total=len(benign_dataframe)) as pbar:
    for indices, batch in batch_dataloader(benign_dataframe,2,prompt='BBox',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, benign_dataframe,path_to_save) 
        pbar.update(2)

In [ ]:
path_to_save = os.path.join('SAM_predictions_BBox','malignant')
with tqdm.tqdm(total=len(malignant_dataframe)) as pbar:
    for indices, batch in batch_dataloader(malignant_dataframe,2,prompt='BBox',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, malignant_dataframe, path_to_save)
        pbar.update(2)

In [ ]:
# clear vram 
clear_vram()

## only Point prompt

In [ ]:
path_to_save = os.path.join('SAM_predictions_Point','benign')
with tqdm.tqdm(total=len(benign_dataframe)) as pbar:
    for indices, batch in batch_dataloader(benign_dataframe,2,prompt='Point',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, benign_dataframe,path_to_save) 
        pbar.update(2)

In [ ]:
path_to_save = os.path.join('SAM_predictions_Point','malignant')
with tqdm.tqdm(total=len(malignant_dataframe)) as pbar:
    for indices, batch in batch_dataloader(malignant_dataframe,2,prompt='Point',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, malignant_dataframe, path_to_save)
        pbar.update(2)

In [ ]:
# clear vram 
clear_vram()

## BBox+Point prompt

In [ ]:
path_to_save = os.path.join('SAM_predictions_BBoxPoint','benign')
with tqdm.tqdm(total=len(benign_dataframe)) as pbar:
    for indices, batch in batch_dataloader(benign_dataframe,2,prompt='BBox+Point',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, benign_dataframe,path_to_save) 
        pbar.update(2)

In [ ]:
path_to_save = os.path.join('SAM_predictions_BBoxPoint','malignant')
with tqdm.tqdm(total=len(malignant_dataframe)) as pbar:
    for indices, batch in batch_dataloader(malignant_dataframe,2,prompt='BBox+Point',shuffle=False):
        with torch.no_grad():
            batch_op = sam_model(batch, multimask_output = False)
            clear_vram()
            unpack_and_save_op(indices, batch_op, malignant_dataframe, path_to_save)
        pbar.update(2)

In [ ]:
# clear vram 
clear_vram()